# Gradient Boosting

In [11]:
from sklearn.ensemble import GradientBoostingClassifier
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold

import numpy as np
import pandas as pd

model = GradientBoostingClassifier(random_state=42)
kf = KFold(n_splits=5, shuffle=True, random_state=42)
df = pd.read_csv('..\\data\\data.csv')

df = df[df['concentration'] > 0]

df.to_csv('..\\data\\data_cleaned.csv', index=False)


# Definisco le variabili feature
features = ['wind_direction', 'concentration', 'Wind Speed', 'azimuth_degrees']
X = df[features]
# Definisco la variabile target
y = df['distance']


In [14]:
from sklearn.impute import SimpleImputer
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [16]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)

xgb_model = XGBClassifier(tree_method='gpu_hist', predictor='gpu_predictor', random_state=42)
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.8, 1.0],
}

grid = GridSearchCV(
    xgb_model,
    param_grid,
    cv=5,
    scoring='accuracy',
    verbose=2,
    n_jobs=-1
)

imputer = SimpleImputer(strategy='constant', fill_value=0)
X_train_imputed = imputer.fit_transform(X_train)

grid.fit(X_train_imputed, y_train_enc)
print("Best parameters found: ", grid.best_params_)
print("Best cross-validation score: ", grid.best_score_)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


c:\Users\abbon\miniconda3\envs\DA\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:41:45] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\abbon\miniconda3\envs\DA\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:41:45] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "predictor" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Best parameters found:  {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'subsample': 0.8}
Best cross-validation score:  0.6665259218783633
